## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
#Find current working directory

#city_data_df = pd.read_csv("WeatherPy_Database.csv")
#city_data_df.head()

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
0,0,Saint Anthony,45.0205,-93.2180,33.80,69,90,9.22,US,1/31/2021 22:55,overcast clouds
1,1,Nikolskoye,59.7035,30.7861,26.60,86,75,6.71,RU,1/31/2021 22:55,snow
2,2,Rikitea,-23.1203,-134.9692,79.25,62,29,7.14,PF,1/31/2021 22:55,scattered clouds
3,3,Kimbe,-5.5502,150.1429,83.46,76,85,6.31,PG,1/31/2021 22:55,light rain
4,4,Takaka,-40.8500,172.8000,66.00,79,0,3.00,NZ,1/31/2021 22:55,clear sky


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Country                 object
Date                    object
Weather Description     object
dtype: object

In [4]:
city_data_df['Country'] = city_data_df['Country'].astype('str')
city_data_df['Weather Description'] = city_data_df['Weather Description'].astype('str')

In [5]:
city_data_df.dtypes

City_ID                  int64
City                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Country                 object
Date                    object
Weather Description     object
dtype: object

In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
2,2,Rikitea,-23.1203,-134.9692,79.25,62,29,7.14,PF,1/31/2021 22:55,scattered clouds
3,3,Kimbe,-5.5502,150.1429,83.46,76,85,6.31,PG,1/31/2021 22:55,light rain
6,6,Saint-Philippe,-21.3585,55.7679,80.60,74,20,12.66,RE,1/31/2021 22:55,light rain
9,9,Brufut,13.3814,-16.7518,78.80,38,0,12.15,GM,1/31/2021 22:55,clear sky
11,11,Iquique,-20.2208,-70.1431,73.40,60,0,20.71,CL,1/31/2021 22:53,clear sky
15,15,Kapaa,22.0752,-159.3190,78.80,61,1,11.50,US,1/31/2021 22:50,clear sky
17,17,Cayenne,4.9333,-52.3333,80.60,78,75,6.91,GF,1/31/2021 22:55,broken clouds
21,21,Hithadhoo,-0.6000,73.0833,82.20,77,75,13.33,MV,1/31/2021 22:55,broken clouds
25,25,Faanui,-16.4833,-151.7500,81.54,75,78,10.36,PF,1/31/2021 22:55,light rain
26,26,Mataura,-46.1927,168.8643,72.37,43,47,1.97,NZ,1/31/2021 22:55,scattered clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                254
City                   254
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Country                254
Date                   254
Weather Description    254
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.25,scattered clouds,-23.1203,-134.9692,
3,Kimbe,PG,83.46,light rain,-5.5502,150.1429,
6,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,
9,Brufut,GM,78.80,clear sky,13.3814,-16.7518,
11,Iquique,CL,73.40,clear sky,-20.2208,-70.1431,
15,Kapaa,US,78.80,clear sky,22.0752,-159.3190,
17,Cayenne,GF,80.60,broken clouds,4.9333,-52.3333,
21,Hithadhoo,MV,82.20,broken clouds,-0.6000,73.0833,
25,Faanui,PF,81.54,light rain,-16.4833,-151.7500,
26,Mataura,NZ,72.37,scattered clouds,-46.1927,168.8643,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
 


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# Print out hotel df 
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.25,scattered clouds,-23.1203,-134.9692,Pension Maro'i
3,Kimbe,PG,83.46,light rain,-5.5502,150.1429,Kimbe Bay Hotel
6,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
9,Brufut,GM,78.80,clear sky,13.3814,-16.7518,Sheraton Gambia Hotel Resort & Spa
11,Iquique,CL,73.40,clear sky,-20.2208,-70.1431,Backpacker`s Hostel Iquique
15,Kapaa,US,78.80,clear sky,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
17,Cayenne,GF,80.60,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
21,Hithadhoo,MV,82.20,broken clouds,-0.6000,73.0833,Scoop Guest House
25,Faanui,PF,81.54,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
26,Mataura,NZ,72.37,scattered clouds,-46.1927,168.8643,Ellie's Villa


In [13]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN") 
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)



               City Country  Max Temp Weather Description      Lat       Lng  \
2           Rikitea      PF     79.25    scattered clouds -23.1203 -134.9692   
3             Kimbe      PG     83.46          light rain  -5.5502  150.1429   
6    Saint-Philippe      RE     80.60          light rain -21.3585   55.7679   
9            Brufut      GM     78.80           clear sky  13.3814  -16.7518   
11          Iquique      CL     73.40           clear sky -20.2208  -70.1431   
..              ...     ...       ...                 ...      ...       ...   
698       Coihaique      CL     78.80          few clouds -45.5752  -72.0662   
699    Todos Santos      MX     70.00       broken clouds  23.4500 -110.2167   
706        Adelaide      AU     71.01          light rain -34.9333  138.6000   
710        Kirakira      SB     84.92       broken clouds -10.4544  161.9205   
712           Dakar      SN     75.20           clear sky  14.6937  -17.4441   

                             Hotel Name

In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.25,scattered clouds,-23.1203,-134.9692,Pension Maro'i
3,Kimbe,PG,83.46,light rain,-5.5502,150.1429,Kimbe Bay Hotel
6,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
9,Brufut,GM,78.80,clear sky,13.3814,-16.7518,Sheraton Gambia Hotel Resort & Spa
11,Iquique,CL,73.40,clear sky,-20.2208,-70.1431,Backpacker`s Hostel Iquique
15,Kapaa,US,78.80,clear sky,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
17,Cayenne,GF,80.60,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
21,Hithadhoo,MV,82.20,broken clouds,-0.6000,73.0833,Scoop Guest House
25,Faanui,PF,81.54,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
26,Mataura,NZ,72.37,scattered clouds,-46.1927,168.8643,Ellie's Villa


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))